In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

#plotly imports
#!pip install plotly==5.10.0
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
pio.renderers.default = "colab"

# 
from scipy import stats
from scipy.spatial.distance import cdist

# Access data using Google BigQuery.
from google.colab import auth, drive
from google.cloud import bigquery

import warnings
warnings.filterwarnings('ignore')

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [36]:
# authenticate
auth.authenticate_user()

# mount google drive
drive.mount('/content/drive')
DATA_DIR = 'drive/MyDrive/COMP90089/group_project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# Set up environment variables
project_id = 'comp90089'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

In [38]:
##Module ICU stay with VITAL sign, gender, age
df_Glucose = run_query("""
SELECT 
  LAB.subject_id, LAB.stay_id, LAB.glucose_max, 
  DEMO.anchor_age, DEMO.dod, DEMO.gender, 
  VITAL.dbp_mean, VITAL.sbp_mean, VITAL.glucose_mean, VITAL.heart_rate_mean, 
  VITAL.spo2_mean, VITAL.resp_rate_mean, VITAL.temperature_mean, 
  IND.apsiii, IND.glucose_score, ICUSTAY.los
FROM `physionet-data.mimiciv_derived.first_day_lab` AS LAB
INNER JOIN `physionet-data.mimiciv_hosp.patients` AS DEMO
ON LAB.subject_id = DEMO.subject_id
INNER JOIN `physionet-data.mimiciv_derived.first_day_vitalsign` AS VITAL
ON LAB.stay_id = VITAL.stay_id
INNER JOIN `physionet-data.mimiciv_derived.apsiii` AS IND
ON LAB.stay_id = IND.stay_id
INNER JOIN `physionet-data.mimiciv_icu.icustays` ICUSTAY
ON LAB.stay_id = ICUSTAY.stay_id
WHERE LAB.glucose_max >= 200
""")
df_Glucose.head()

,subject_id,stay_id,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,los
0,17145022,36446667,256.0,34,NaT,M,66.50,112.166667,210.842105,79.518519,NaN,16.111111,36.310000,52,5,1.092477
1,11693046,31636456,414.0,27,NaT,M,NaN,NaN,254.333333,NaN,NaN,NaN,NaN,20,5,10.672859
2,10122346,30908451,273.0,36,NaT,M,NaN,NaN,121.500000,NaN,NaN,NaN,NaN,9,3,3.615185
3,18818535,38793668,275.0,29,NaT,F,77.00,156.730769,282.857143,93.160000,NaN,22.923077,36.741667,51,5,1.111238
4,12041046,31249096,277.0,36,NaT,M,64.32,112.880000,221.272727,101.800000,92.0,17.346154,37.186250,38,3,1.938414


In [39]:
df_bmi = run_query("""
SELECT subject_id, 
  AVG(CAST(result_value AS FLOAT64)) AS avg_bmi_value
FROM `physionet-data.mimiciv_hosp.omr` 
WHERE result_name="BMI (kg/m2)"
GROUP BY subject_id, result_name
""")
df_bmi.head()
df_bmi.describe()

,subject_id,avg_bmi_value
count,1.448610e+05,144861.000000
mean,1.501533e+07,31.508083
std,2.887037e+06,302.442950
min,1.000003e+07,0.000000
25%,1.250414e+07,23.700000
50%,1.502614e+07,27.226667
75%,1.751656e+07,31.720000
max,1.999983e+07,107840.200000


In [40]:
data_List = pd.DataFrame()

data_List = pd.merge(df_Glucose, df_bmi, on=["subject_id"], how = "left")

data_List.head()
data_List.describe()

,subject_id,stay_id,glucose_max,anchor_age,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,los,avg_bmi_value
count,1.484900e+04,1.484900e+04,14849.000000,14849.000000,14754.000000,14754.000000,14705.000000,14826.000000,14791.000000,14818.000000,14402.000000,14849.000000,14849.000000,14849.000000,9427.000000
mean,1.495145e+07,3.498204e+07,315.499697,63.351337,63.537791,119.606764,269.150999,87.870111,96.691182,19.967010,36.791170,56.653512,3.563607,4.068648,30.558477
std,2.883976e+06,2.883237e+06,173.953706,16.061045,11.643143,18.017578,2885.334677,16.402283,3.122335,4.093006,0.658481,25.267327,0.934916,5.426979,44.926034
min,1.000216e+07,3.000021e+07,200.000000,18.000000,17.000000,45.500000,42.000000,28.500000,26.600000,9.000000,26.670000,5.000000,3.000000,0.001551,3.400000
25%,1.246809e+07,3.248068e+07,223.000000,54.000000,55.431609,106.729823,167.000000,76.175078,95.583333,17.025507,36.572292,39.000000,3.000000,1.197037,24.325000
50%,1.491852e+07,3.497610e+07,260.000000,65.000000,62.666667,117.200000,197.500000,86.924501,97.150000,19.346154,36.805000,51.000000,3.000000,2.186840,27.931818
75%,1.744187e+07,3.745190e+07,334.000000,75.000000,70.440000,130.965209,233.647059,98.666667,98.517880,22.351389,37.078643,69.000000,5.000000,4.541898,33.114286
max,1.999983e+07,3.999775e+07,5840.000000,91.000000,129.571429,217.750000,166855.166667,166.307692,100.000000,40.880000,40.104118,184.000000,9.000000,86.312488,2332.250000


In [41]:
data_List.columns

Index(['subject_id', 'stay_id', 'glucose_max', 'anchor_age', 'dod', 'gender',
       'dbp_mean', 'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean',
       'resp_rate_mean', 'temperature_mean', 'apsiii', 'glucose_score', 'los',
       'avg_bmi_value'],
      dtype='object')

In [42]:
# Fill null for BMI
data_List['avg_bmi_value'] = data_List['avg_bmi_value'].fillna(
                                      data_List['avg_bmi_value'].mean())

# One-hot Encoding: Female = 1, Male = 0
data_List['gender'] = pd.get_dummies(data_List['gender']) ['F'] 

# if dod is NAN, replace with 0, if dod has a value, replace with 1
data_List['dod'] = data_List['dod'].notnull().astype("int") 

In [43]:
## Copy data for q1 analysis
df = data_List.copy()


## For Non-numberical columns (gender, dod): count, unique, top, freq
df.describe(include='all')

## Define functions to Remove outliers ##
def remove_outliers(df, col_lst):
  for col in col_lst:
    # we remove items that are outside of 99.5% of quantile 
    lower_q, higher_q = df[col].quantile(0.005), df[col].quantile(0.995)
    filtered_df = df[(df[col] > lower_q) & (df[col] < higher_q)]

  return filtered_df    

df_filtered = remove_outliers(df, ['glucose_max', 'anchor_age',
       'dbp_mean', 'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean',
       'resp_rate_mean', 'temperature_mean', 'los',
       'avg_bmi_value'] )

df_final = df_filtered.dropna() # Drop Nulls
df_final.describe(include='all')

# df_final.to_csv(index=False)


,subject_id,stay_id,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,los,avg_bmi_value
count,1.407700e+04,1.407700e+04,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000,14077.000000
mean,1.496148e+07,3.497575e+07,315.593024,63.346594,0.494139,0.458834,63.603645,119.974062,268.468660,87.898336,96.750843,19.912943,36.794914,56.276977,3.564254,4.016762,29.744926
std,2.884766e+06,2.884496e+06,174.402243,16.034933,0.499983,0.498320,11.538817,17.766778,2927.594228,16.266383,2.766991,4.045583,0.646371,25.020672,0.935059,5.275067,5.552130
min,1.000216e+07,3.000021e+07,200.000000,18.000000,0.000000,0.000000,17.000000,50.863636,42.000000,28.500000,51.250000,9.916667,30.666667,5.000000,3.000000,0.016979,17.000000
25%,1.247542e+07,3.246231e+07,223.000000,54.000000,0.000000,0.000000,55.538462,107.000000,167.111111,76.238095,95.600000,17.000000,36.573333,39.000000,3.000000,1.229317,26.375000
50%,1.494436e+07,3.496584e+07,260.000000,65.000000,0.000000,0.000000,62.708333,117.571429,197.478261,87.000000,97.136364,19.312500,36.805000,50.000000,3.000000,2.183646,30.558477
75%,1.744981e+07,3.744717e+07,334.000000,75.000000,1.000000,1.000000,70.440000,131.166667,233.250000,98.680000,98.500000,22.269231,37.080000,68.000000,5.000000,4.433576,30.558477
max,1.999983e+07,3.999775e+07,5840.000000,91.000000,1.000000,1.000000,126.458333,217.750000,166855.166667,158.269231,100.000000,40.880000,40.104118,184.000000,9.000000,86.312488,57.300000


In [44]:
# Exporting csv files to local repo 
df_final.to_csv(DATA_DIR+'hyperglycemic_patients.csv', index=False)